In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_3aTBe3_QBE6WK7Q97V5Rb7VrAtrwDUFpVGzaNhHxNgJgwSqJduuQAE7bt2SQgmRxaPGG1Y") # "New API key as of 4/18/2025"

index = pc.Index(name="reddit-shopper", host="https://reddit-shopper-thxz7zo.svc.aped-4627-b74a.pinecone.io")  # the name you gave it


In [27]:
search_with_text = index.search(
    namespace="reddit-namespace", 
    query={
        "inputs": {"text": "mother"}, 
        "top_k": 4
    },
    fields=["category", "chunk_text"],
    rerank={
        "model": "bge-reranker-v2-m3",
        "top_n": 2,
        "rank_fields": ["chunk_text"] # Specified field must also be included in 'fields'
    }
)

print(search_with_text)

{'result': {'hits': [{'_id': 'rec281',
                      '_score': 0.06804041564464569,
                      'fields': {'category': 'I Just Biked Across the Bolivian '
                                             'Altiplano',
                                 'chunk_text': 'Beauty'}},
                     {'_id': 'rec1022',
                      '_score': 0.056861549615859985,
                      'fields': {'category': 'Dead Vlei Namibia. Believe me or '
                                             'not but I hitchhiked to this '
                                             'place back in 2017',
                                 'chunk_text': 'Furiosa'}}]},
 'usage': {'embed_total_tokens': 4, 'read_units': 6, 'rerank_units': 1}}


In [ ]:
%%writefile test_app.py
from pinecone import Pinecone
import streamlit as st

# Init Pinecone
pc = Pinecone(api_key="pcsk_3aTBe3_QBE6WK7Q97V5Rb7VrAtrwDUFpVGzaNhHxNgJgwSqJduuQAE7bt2SQgmRxaPGG1Y")
index = pc.Index("reddit-shopper")

# Streamlit UI
st.title("📘 Semantic Reddit Comment Search")

query = st.text_input("Search for a product or experience", "mountain")

if st.button("Search"):
    try:
        search_with_text = index.search(
            namespace="reddit-namespace",
            query={
                "inputs": {"text": query},
                "top_k": 5
            },
            fields=["category", "chunk_text"],
            rerank={
                "model": "bge-reranker-v2-m3",
                "top_n": 3,
                "rank_fields": ["chunk_text"]
            }
        )

        results = search_with_text.result.get("hits", [])
        if not results:
            st.warning("No results found.")
        else:
            st.subheader("🔍 Top Matching Comments")
            for item in results:
                fields = item.get("fields", {})
                st.markdown(f"**Score:** {item['_score']:.3f}")
                st.markdown(f"**Category:** {fields.get('category', 'N/A')}")
                st.markdown(f"**Comment:** {fields.get('chunk_text', 'No text found')}")
                st.markdown("---")

    except Exception as e:
        st.error(f"Search error: {e}")




Overwriting app.py


In [33]:
!streamlit run app.py


^C
